# Explore a NuGrid star model in iPython notebook

Modified for ASTR 404, Winter 2024. Execute this notebook on the [Astohub](https://astrohub.uvic.ca).


### Initialize session

In [ ]:
%pylab ipympl
from nugridpy import mesa as ms
from nugridpy import nugridse as mp

data_dir="/data/ASDR/NuGrid"  
# data_dir = '/Users/fherwig/mnt/nugrid'

ms.set_nugrid_path(data_dir)
mp.set_nugrid_path(data_dir)

### The MESA stellar evolution model

By default MESA is putting out two types of data. History data provides the time evolution of scalar quantities, one per time step. This data can be accessed with the `mesa.star_log` (or `mesa.history_data` which is the same) class.

MESA also outputs profile data at select time steps. Profiles are available via the `mesa_profile` class.

#### History data
Initialise the 2 solar-mass Z=0.02 MESA stellar evolution model from set1.2 using the seeker method:

In [ ]:
s=ms.star_log(mass=1,Z=0.02)   # or ms.history_data which is just an alias

`s` is an instance of the history data, which is the table of scalar quantities such as mass, $T_\mathrm{eff}$ etc for each time step or model.

There are a number of pre-defined functions or methods associated with this instances, such as an HRD or a Kippenhahn diagram:

In [ ]:
ifig=102;close(ifig);figure(ifig)
s.hrd_new()
legend(loc='lower right').draw_frame(False)

In [ ]:
ifig=107;close(ifig);figure(ifig)
s.kip_cont(ifig=ifig, boundaries=True,engenlevels=[1e2,1e6,1e12],modstart=100,modstop=500)

Note that each function has a pythonic doc string, e.g. `s.kip_cont`. 

The `s.cols` command gives a list of available columns in the history file. Maybe the most important method is `s.get('col_name)` which returns an array with the data in the column with name `col_name`.

Below we demonstrate this by funding a model similar to that of our sun (age $\approx 4.6$ billion years, mass $1\mathrm{M_\odot}$ and $Z=0.02$ and then find the central temperature and pressure of that stellar model.

In [ ]:
# this is the array with the ages of the stellar models
s.get('star_age')

In [ ]:
# find the index that corresponds to the approximate age of the sun
# recall that we initiated `s` as a 1Msun, Z=0.02 model
ind_sun = where(s.get('star_age')<4.6e9)[0][-1]
ind_sun

In [ ]:
# let's check what quantities we have
s.cols

As we can see we have the central temperature as well as some other central quantities already listed as scalar quantities in the history file:

```
'log_center_T': 123,
 'log_center_Rho': 124,
 'log_center_P': 125,
 'center_mu': 126,
 'center_ye': 127,
 'center_h1': 128
 ```
 
 Therefore knowing the central T and P is straight forward:

In [ ]:
s.get('log_center_T')[ind_sun]

In [ ]:
s.get('log_center_P')[ind_sun]

In [ ]:
s.get('center_h1')[ind_sun]

We can find the same results also by looking at profile data from the MESA output.

#### MESA profile files
The `mesa.mesa_profile` class provides access to the available profile data. We can again use the seeker method to just find the righ model by initial mass and metallicity. An additional argument `num=` specifies the model number. This behaviour can be changed, see the doc string for details. Profiles are not available for all models, and the method will report which nearby model it has found.

In [ ]:
# The `prof` instance is now holding the data of one profile for the given model number in the `num=` argument.
prof=ms.mesa_profile(mass=1,Z=0.02,num=ind_sun)

In [ ]:
# Again, have a look at the available quantieis
prof.cols

In [ ]:
# The data is presented in arrays that go from the outside inward
# therefore the last element is the central value
prof.get('pressure')[-1]

In [ ]:
# Of course we can also plot profiles
ifig=108;close(ifig);figure(ifig)
plot(prof.get('rmid'),prof.get('mu'))

In [ ]:
ifig=108;close(ifig);figure(ifig)
prof.plot('mass','c12',shape='-')

In [ ]:
# get all header attributes, these are some scalar quantites (like the ones in the history instance)
prof.header_attr

The MESA data has a number of abundance profiles saved, but not all. These are available from the nucleosynthesis post-processing simulation with the `mppnp` code.

## Analyse the `mppnp` post-processing data

In the above example at model `500` the $1\mathrm{M_\odot}$ model star is in the Red Giant Branch phase with an inert He core and supported by H-shell burning. It has an extended convective envelope. Let's look at the abundance profiles at this stage. For this we are using the post-processing data. Just make sure you are selecting the same mass and Z value.

In [ ]:
pt=mp.se(mass=1,Z=0.02)

#### What quantities are available in this data type, and what are the units?
Each of the _se_ file sets (in fact each of the dozens of hdf5 files that make up the data set for one mass/metallicty combination, or stellar evolution track) has three types of data contained in them:

data type access | content 
----------------|---------
 `pt.se.hattrs` |  a header section that holds the _header attributes_, including units in the form of factors so that if applied with the quantities the result is in cgs units 
`pt.se.cattrs` | for each cycle (or time step) the _cycle attributes_ are a number of scalar global quantities, such as total mass or star age
`pt.se.dcols` | again, for each time step these are the vector quantities available, i.e. the data table columns; one of the data columns, _iso_massf_ is in fact a matrix where the matrix columns are different species, i.e. a radial vector of species vectors

We can use the `get` method to pull any of the data. But for some standard plots there are pre-made methods available, such as for plotting the profiles of a number of isotopes. This method is the `pt.abu_profile` method. Have a look at the doc string.

A complete list of isotopes can be optained with the command 
```Python
pt.se.isotopes
```

In [ ]:
pt.abu_profile?

In [ ]:
species=['H-1','He-4','C-12','C-13','N-14','N-15','O-16']
ifig=108;close(ifig);figure(ifig)
pt.abu_profile(isos=species, ifig=ifig, fname=500, logy=True)
ylim(-7,0)
legend(loc='lower right',fancybox=True, framealpha=0.9).get_frame().set_linewidth(0.0)

The interesting features are at mass coordinate $\approx 0.255 M_{\odot}$. Let's have a look at this in more detail.

In [ ]:
species=['H-1','He-4','C-12','C-13','N-14','N-15','O-16']
ifig=109;close(ifig);figure(ifig)
pt.abu_profile(isos=species, ifig=ifig, fname=500, logy=True, colourblind=True)
ylim(-7,0)
xlim(0.242,0.260)
legend(loc='lower left',fancybox=True, framealpha=0.9).get_frame().set_linewidth(0.0)

Explain the abundance profiles that you see in the above plot in terms of the CN cycle H-burning.

A chart plot of a certain N and Z range of the chart of isotopes may help. Provide a radial mass range for which the abundances will be averaged to provide the charts:

In [ ]:
# pt.abu_chart?

In [ ]:
close(500)
pt.abu_chart(500,mass_range=(0.2525,0.2535),plotaxis=[0,24,0,24])

## Using the astropy constants and units package


In [2]:
from astropy import units as uu
from astropy import constants as cc

### Temperature corresponding to complete ionization of H

In [5]:
# internal energy of a gas of hydrogen atoms at 10^4 K
E_int = (3./2) * cc.k_B * 1e4 * uu.K
print(E_int.to('erg'))

2.0709735e-12 erg


In [6]:
# ionization energy of hydrogen
E_ionis_H = 13.6 * uu.eV
print(E_ionis_H.to('erg'))

2.1789602222399996e-11 erg


In [7]:
# temperature corresponding to E_ionis_H
T_ionis_H = (E_ionis_H / (1.5 * cc.k_B)).to('K')
print(f"{T_ionis_H:.2e}")

1.05e+05 K


In [9]:
# binding energy of the sun
mass_sun = 1 * uu.M_sun
R_sun = 1 * uu.R_sun
G = cc.G
E_bind_sun = (3 * G * mass_sun**2) / (5 * R_sun)
print(E_bind_sun.to('erg'))

# assume binding energy of 1Msun white dwarf with 0.01 Rsun is released over 100 days, calculate the luminosity
mass_wd = 1 * uu.M_sun
R_wd = 0.01 * uu.R_sun
t = 100 * uu.day
E_bind_wd = (3 * G * mass_wd**2) / (5 * R_wd)
L_wd = E_bind_wd / t
print(L_wd.to('L_sun'))

2.275866542996316e+48 erg
6881151502.441548 solLum
